In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import random


from transformers import BertTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

movies = pd.read_json('IMDB_movie_details.json',lines=True)
movies

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
...,...,...,...,...,...,...,...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."
1569,tt5013056,Evacuation of Allied soldiers from the British...,1h 46min,"[Action, Drama, History]",8.1,2017-07-21,The film alternates between three different pe...
1570,tt0104014/,"For a while now, beautiful 24-year-old Diana B...",1h 33min,"[Comedy, Drama]",5.3,1992-02-21,


In [2]:
anime = pd.read_csv('animes.csv')
anime = anime.drop_duplicates(subset=['title'])
anime

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
...,...,...,...,...,...,...,...,...,...,...,...,...
19002,10075,Naruto x UT,All-new animation offered throughout UNIQLO cl...,"['Action', 'Comedy', 'Super Power', 'Martial A...","Jan 1, 2011",1.0,34155,2382,1728.0,7.50,https://cdn.myanimelist.net/images/anime/3/304...,https://myanimelist.net/anime/10075/Naruto_x_UT
19003,35828,Miira no Kaikata,High school student Sora Kashiwagi is accustom...,"['Slice of Life', 'Comedy', 'Supernatural']","Jan 12, 2018 to Mar 30, 2018",12.0,61459,1648,1727.0,7.50,https://cdn.myanimelist.net/images/anime/1486/...,https://myanimelist.net/anime/35828/Miira_no_K...
19004,10378,Shinryaku!? Ika Musume,"After regaining her squid-like abilities, Ika ...","['Slice of Life', 'Comedy', 'Shounen']","Sep 27, 2011 to Dec 25, 2011",12.0,67422,1547,1548.0,7.56,https://cdn.myanimelist.net/images/anime/6/301...,https://myanimelist.net/anime/10378/Shinryaku_...
19005,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,https://cdn.myanimelist.net/images/anime/12/79...,https://myanimelist.net/anime/33082/Kingsglaiv...


In [3]:
import os
from html import unescape

substrcheck = np.vectorize(lambda a,A: a in A,otypes=[bool])

def character_shows(character_html):
    Shows = np.array(character_html.split('</a>'))
    Shows = np.vectorize(lambda a: a.split(">")[-1],otypes=[str])(Shows[np.where(substrcheck('<td valign="top" class="borderClass">',Shows))])
    return np.unique(Shows)

def apply_to(thing,folder,D,fun):
    try:
        with open(folder+"/"+thing,"r",encoding='utf-8') as f:
            D[thing.split(".")[0]] = fun(unescape(f.read()))
    except PermissionError:
        pass

fire_shows = {}
firent_shows = {}

np.vectorize(apply_to,otypes=[None])(np.array(os.listdir('fire users')),'fire users',fire_shows,character_shows)
np.vectorize(apply_to,otypes=[None])(np.array(os.listdir('firent users')),'firent users',firent_shows,character_shows)
fire_shows

{'Kilik Lunge': array(['Soul Eater', 'Soul Eater NOT!'], dtype='<U15'),
 'Shigekuni Yamamoto-Genryuusai': array(['Bleach', 'Bleach KaraBuri!: Gotei Juusan Yatai Daisakusen!',
        'Bleach Movie 1: Memories of Nobody',
        'Bleach Movie 2: The DiamondDust Rebellion - Mou Hitotsu no Hyourinmaru',
        'Bleach Movie 3: Fade to Black - Kimi no Na wo Yobu',
        'Bleach Movie 4: Jigoku-hen', 'Bleach: Sennen Kessen-hen',
        'Bleach: Sennen Kessen-hen - Ketsubetsu-tan',
        'Bleach: Sennen Kessen-hen - Soukoku-tan',
        'Bleach: Spirits Are Forever with You',
        'Bleach: The Death Save the Strawberry',
        'Gekijouban Bleach: Memories of Nobody',
        'Tsurezure Nichijou Ekotoba Colourful Bleach'], dtype='<U70'),
 '(MC) Charizard': array(['A Ripple in Time', 'CoroCoro Monster Ball', 'Dengeki! Pikachu',
        'Gekijouban Pocket Monster Best Wishes: Shinsoku no Genesect - Mewtwo Kakusei',
        'Gekijouban Pocket Monsters: Kimi ni Kimeta!',
        'Gek

In [4]:
fire_mc = np.array([])
fire_mcnt = np.array([])
firent = np.array([])

for show in fire_shows.items():
    if "(MC)" in show[0]:
        fire_mc = np.concatenate((fire_mc,show[1]))
    else:
        fire_mcnt = np.concatenate((fire_mc,show[1]))

for show in firent_shows.items():
    firent = np.concatenate((firent,show[1]))

fire_mc = np.unique(fire_mc)
fire_mcnt = np.unique(fire_mcnt)
firent = np.unique(firent)
fire_mc

array(['A Ripple in Time', 'Ao no Exorcist', 'Ao no Exorcist Movie',
       'Ao no Exorcist Movie Special', 'Ao no Exorcist: Kuro no Iede',
       'Ao no Exorcist: Kyoto Fujouou-hen',
       'Ao no Exorcist: Kyoto Fujouou-hen OVA',
       'Ao no Exorcist: Shimane Illuminati-hen', 'Ao no Exorcist: Ura Ex',
       'Ao no Exorcist: Yuki no Hate-hen', 'Bakugan Battle Brawlers',
       'Bakugan Battle Brawlers: Gundalian Invaders',
       'Bakugan Battle Brawlers: Mechtanium Surge',
       'Bakugan Battle Brawlers: New Vestroia', 'Beyblade Burst',
       'Beyblade Burst Chouzetsu', 'Beyblade Burst Gachi',
       'Beyblade Burst Sparking', 'CoroCoro Monster Ball',
       'Dengeki Bunko 2007 Movie Festival Special', 'Dengeki! Pikachu',
       'Digimon Adventure', 'Digimon Adventure 02',
       'Digimon Adventure 02 Movies',
       'Digimon Adventure 02: Diablomon no Gyakushuu',
       'Digimon Adventure 3D: Digimon Grand Prix!',
       'Digimon Adventure Movie', 'Digimon Adventure tri. 1: Sai

In [5]:
fire_mc_synopsis = {}
fire_mcnt_synopsis = {}
firent_synopsis = {}

stopcheck = np.vectorize(lambda w: w in stopwords.words('english'),otypes=[bool])

def fill_synopsis(title,D,label):
    try:
        D[title] = np.array(np.array(anime[anime['title']==title]['synopsis'])[0].replace('(Source:','[Written by MAL Rewrite]').split('[Written by MAL Rewrite]')[0].replace('\r\n',' ').replace('.','').replace(',','').replace('!','').replace('"','').replace('(','').replace(')','').lower().split(' '))
        D[title] = np.delete(D[title],np.where(D[title]==''))
        D[title] = np.delete(D[title],np.where(stopcheck(D[title])))
        D[title] = ' '.join(D[title])
        while '  ' in D[title]:
            D[title] = D[title].replace('  ',' ')
        D[title] = [D[title],label]
    except:
        pass

np.vectorize(lambda p: fill_synopsis(p,fire_mc_synopsis,1))(fire_mc)
np.vectorize(lambda p: fill_synopsis(p,fire_mcnt_synopsis,1))(fire_mcnt)
np.vectorize(lambda p: fill_synopsis(p,firent_synopsis,0))(firent)
fire_mc_synopsis

{'Ao no Exorcist': ['humans demons two sides coin assiah gehenna respective worlds way travel realms means possession like ghost stories however satan ruler gehenna cannot find suitable host possess therefore remains imprisoned world desperate attempt conquer assiah sends son instead intending eventually grow vessel capable possession demon king ao exorcist follows rin okumura appears ordinary somewhat troublesome teenager—that one day ambushed demons world turns upside discovers fact son satan demon father wishes return conquer assiah together wanting join king gehenna rin decides begin training become exorcist fight defend assiah alongside brother yukio',
  1],
 'Ao no Exorcist Movie': ['atmosphere true cross academy lively boisterous days leading grand festival held every 11 years time okumura rin entrusted responsibility suppressing berserk phantom train midst mission meets devil whose appearance young boy',
  1],
 'Ao no Exorcist Movie Special': ['collaboration blue exorcist kakko

In [6]:
from nltk.metrics.distance import jaccard_distance

Dragon_Cry = fire_mc_synopsis['Fairy Tail Movie 2: Dragon Cry'][0]
Phoenix_Priestess = fire_mc_synopsis['Fairy Tail Movie 1: Houou no Miko'][0]

1-jaccard_distance(set(Phoenix_Priestess),set(Dragon_Cry))

0.7931034482758621

In [7]:
fire_mc_synopsis2 = {}
fire_mcnt_synopsis2 = {}

# reducing sample size
for k in random.sample(list(fire_mc_synopsis.keys()),50):
    fire_mc_synopsis2[k] = fire_mc_synopsis[k]
for k in random.sample(list(fire_mcnt_synopsis.keys()),50):
    fire_mcnt_synopsis2[k] = fire_mcnt_synopsis[k]

fire_mc_synopsis = fire_mc_synopsis2
fire_mcnt_synopsis = fire_mcnt_synopsis2
fire_mc_synopsis

{'Digimon Savers': ['masaru second year junior high student undefeated street-fighter learn meaning word teamwork agumon escapes dats organization deals digimon winds fight masaru soon yoshino lalamon come capture agumon two fighters escape evil digimon appears masaru agumon defeat dna charge soon masaru must join dats put end digimon coming earth help yoshino ikuto touma group must stop evil digimon appear return digital world everything goes planned',
  1],
 'Digimon Adventure Movie': ['brother sister discover digital world 1s 0s living creature arrives family computer adventures group children start appearance digital monster real world',
  1],
 'Digimon Adventure 02 Movies': ['years young boy america loses one digimon friends evil viral digimon successfully kidnaps original chosen children younger friends must race time digimon partners discover source new menace perhaps solve years-old mystery',
  1],
 'Bakugan Battle Brawlers: Gundalian Invaders': ['dan marucho shun get caught wa

In [8]:
tfidf = TfidfVectorizer(stop_words=stopwords.words('english'))
firewords = tfidf.fit_transform(list([f[0] for f in fire_mc_synopsis.values()]))
tfidf.get_feature_names_out()

array(['02', '0s', '10', ..., 'zash', 'zukan', 'éclair'], dtype=object)

In [9]:
firewords.shape

(50, 1490)

In [10]:
# code copied from Chi

class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_dim, n_layers, bidirectional, dropout):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out[:, -1, :])
        return output
    

# Training function
def train_lstm(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    total_acc = 0

    # Wrap the dataloader with tqdm for progress tracking
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)

        # Forward pass
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        preds = torch.argmax(outputs, dim=1)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

    return total_loss / len(dataloader), total_acc / len(dataloader)


# Evaluation function
def evaluate_lstm(model, dataloader):
    model.eval()
    total_acc = 0

    with torch.no_grad():
        # Wrap the dataloader with tqdm for progress tracking
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids)
            preds = torch.argmax(outputs, dim=1)

            total_acc += accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

    return total_acc / len(dataloader)

In [11]:
anime_fire = pd.DataFrame.from_dict({**fire_mc_synopsis, **fire_mcnt_synopsis, **firent_synopsis},orient='index')
anime_fire.rename(columns={0: 'synopsis',1: 'has_fire'}, inplace=True)
anime_fire

,synopsis,has_fire
Digimon Savers,masaru second year junior high student undefea...,1
Digimon Adventure Movie,brother sister discover digital world 1s 0s li...,1
Digimon Adventure 02 Movies,years young boy america loses one digimon frie...,1
Bakugan Battle Brawlers: Gundalian Invaders,dan marucho shun get caught war two different ...,1
Fairy Tail Movie 1: Houou no Miko,mountains north fiore lies fire village lush-b...,1
...,...,...
Hori-san to Miyamura-kun,within everyone exists side preferably kept hi...,0
Ouran Koukou Host Club,haruhi fujioka bright scholarship candidate ra...,0
Shingeki no Kyojin,centuries ago mankind slaughtered near extinct...,0
Shingeki no Kyojin Season 2,centuries humanity hunted giant mysterious pre...,0


In [37]:
anime_fire1 = anime_fire[anime_fire['has_fire'] == 1]

In [12]:
# code copied from Chi

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class FireDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = tokenizer.encode_plus(
            text,
            max_length=5000,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

train_ds = FireDataset(anime_fire['synopsis'].tolist(),anime_fire['has_fire'].tolist())

In [22]:
N = 200
moviesample = movies.sample(n=N)

def cleanup(s):
    S = s.replace('\r\n',' ').replace('.','').replace(',','').replace('!','').replace('"','').replace('(','').replace(')','')
    while '  ' in S:
        S = S.replace('  ',' ')
    return S
cleanup = np.vectorize(cleanup,otypes=[str])

moviesample.loc[:,'plot_synopsis'] = cleanup(moviesample.loc[:,'plot_synopsis'])
moviesample['has_fire'] = 0 # this is technically incorrect but we'll check it out later
moviesample

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis,has_fire
69,tt0106977,A well respected Chicago surgeon Dr. Richard K...,2h 10min,"[Action, Crime, Drama]",7.8,1993-08-06,Flashes of the city skyline and fight between ...,0
360,tt1578873,"Set in the fictional town of Rosewood, Pennsyl...",44min,"[Drama, Mystery, Romance]",7.6,2010-07-22,1 year ago Allison the queen bee of the four f...,0
557,tt0116683,James' happy life at the English seaside is ru...,1h 19min,"[Animation, Adventure, Family]",6.7,1996-04-12,,0
35,tt0018455,"In this fable-morality subtitled ""A Song of Tw...",1h 34min,"[Drama, Romance]",8.2,1927-11-04,In the summertime described as vacation time a...,0
108,tt1355644,"The spaceship, Starship Avalon, in its 120-yea...",1h 56min,"[Drama, Sci-Fi, Thriller]",7.0,2016-12-21,The movie opens in outer space with the starsh...,0
252,tt0892769,"Long ago up North on the Island of Berk, the y...",1h 38min,"[Animation, Adventure, Comedy]",8.1,2010-03-26,A Viking boy called Hiccup voice: Jay Baruchel...,0
826,tt0116908,"Samantha Caine, suburban homemaker, is the ide...",2h 1min,"[Action, Crime, Drama]",6.7,1996-10-11,,0
977,tt0108002,Rudy grew up in a steel mill town where most p...,1h 54min,"[Biography, Drama, Sport]",7.5,1993-10-22,Rudy Ruettiger Sean Astin grows up in a Cathol...,0
1059,tt0118747,"Elwood, the now lone ""Blues Brother"" finally r...",2h 3min,"[Action, Comedy, Crime]",4.8,1998-02-06,,0
1376,tt0102558,A financier [Chevy Chase] meets a spurned love...,1h 34min,[Comedy],4.9,1991-02-15,While hosting a party in his Manhattan penthou...,0


In [14]:
moviesample.to_csv('movierngsample{}.csv'.format(N),index=False)

In [15]:
test_ds = FireDataset(moviesample['plot_synopsis'].tolist(),moviesample['has_fire'].tolist())

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False)

embedding_dim = 100
hidden_dim = 128
vocab_size = tokenizer.vocab_size
output_dim = 2
n_layers = 3
bidirectional = True
dropout = 0.5

lstm_model = LSTMClassifier(embedding_dim, hidden_dim, vocab_size, output_dim, n_layers, bidirectional, dropout)
lstm_model = lstm_model.to(device)
optimizer = Adam(lstm_model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [16]:
train_losses = []
train_accuracies = []

n_epochs = 5
# Initialize tqdm progress bar
for epoch in tqdm(range(n_epochs), desc='Training Epochs'):
    train_loss, train_acc = train_lstm(lstm_model, train_loader, optimizer, criterion)
    
    # Append metrics to lists
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # Print epoch information
    tqdm.write(f'Epoch {epoch+1}: Train Loss {train_loss:.4f}, Train Acc {train_acc:.4f}')
    
    # Save model checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        checkpoint_path = f'lstm_model_epoch_{epoch+1}.pth'
        torch.save(lstm_model.state_dict(), checkpoint_path)
        print(f'Model checkpoint saved to {checkpoint_path}')

# Evaluate on test set
test_acc = evaluate_lstm(lstm_model, test_loader)
print(f'Test Accuracy: {test_acc:.4f}')

Training Epochs:  20%|█████▌                      | 1/5 [00:54<03:39, 54.88s/it]

Epoch 1: Train Loss 0.6570, Train Acc 0.5577



Training Epochs:  40%|███████████▏                | 2/5 [01:49<02:43, 54.66s/it]

Epoch 2: Train Loss 0.6540, Train Acc 0.6827



Training Epochs:  60%|████████████████▊           | 3/5 [02:42<01:48, 54.10s/it]

Epoch 3: Train Loss 0.6290, Train Acc 0.6731



Training Epochs:  60%|████████████████▊           | 3/5 [03:33<02:22, 71.21s/it]


KeyboardInterrupt: 

In [25]:
moviesample = pd.read_csv('random data.csv')
moviesample.head(11)

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis,has_fire
0,tt2294449,After making their way through high school (tw...,1h 52min,"['Action', 'Comedy', 'Crime']",7.1,2014-06-13,Following their success in the 21 Jump Street ...,0
1,tt1951264,Twelve months after winning the 74th Hunger Ga...,2h 26min,"['Action', 'Adventure', 'Mystery']",7.5,2013-11-22,Six months after winning the 74th Hunger Games...,0
2,tt0114746,An unknown and lethal virus has wiped out five...,2h 9min,"['Mystery', 'Sci-Fi', 'Thriller']",8.0,1996-01-05,The time is the indeterminate future A virus d...,0
3,tt0112579,The path of Francesca Johnson's future seems d...,2h 15min,"['Drama', 'Romance']",7.6,1995-06-02,NaN,0
4,tt0425112,"Top London cop, PC Nicholas Angel is good. Too...",2h 1min,"['Action', 'Comedy']",7.9,2007-04-20,Nicholas Angel Simon Pegg is undoubtedly Londo...,0
5,tt2184339,In an America wracked by crime and overcrowded...,1h 25min,"['Horror', 'Sci-Fi', 'Thriller']",5.7,2013-06-07,In 2022 the United States is a nation reborn r...,0
6,tt1536537,"In a not so distant future, where overpopulati...",2h 3min,"['Action', 'Adventure', 'Crime']",6.9,2017-08-18,NaN,0
7,tt0107050,John and Max are elderly men living next door ...,1h 43min,"['Comedy', 'Drama', 'Romance']",7.0,1993-12-25,John and Max are elderly neighbors in a cold M...,0
8,tt0050212,The film deals with the situation of British p...,2h 41min,"['Adventure', 'Drama', 'War']",8.2,1957-12-14,In the middle of World War II two prisoners of...,0
9,tt3896198,"After saving Xandar from Ronan's wrath, the Gu...",2h 16min,"['Action', 'Adventure', 'Sci-Fi']",7.8,2017-05-05,Missouri Planet Earth 1980Meredith Quill Laura...,1


In [36]:
import warnings
warnings.filterwarnings("ignore")

In [40]:
anime_fire1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, Digimon Savers to Pokemon Movie 07: Rekkuu no Houmonsha Deoxys
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   synopsis  67 non-null     object
 1   has_fire  67 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


In [43]:
# cosine similarity

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

num_movies = len(moviesample)
num_animes = len(anime_fire1)

# making a matrix
compare_matrix = pd.DataFrame(0, index=range(num_movies), columns=range(num_animes))

for i in range(num_movies):
    for j in range(num_animes):
        movietext = moviesample['plot_synopsis'][i]
        animetext = anime_fire1['synopsis'][j]
        
        if pd.isna(movietext) or pd.isna(animetext):
             compare_matrix.iat[i, j] = 0
        else:
            # Step 1: Preprocess and create term frequency vectors
            vectorizer = CountVectorizer().fit_transform([movietext, animetext])
            vectors = vectorizer.toarray()

            # Step 2: Calculate cosine similarity
            cosine_sim = cosine_similarity(vectors)

            # Store the similarity score in the compare_matrix
            compare_matrix.iat[i, j] = cosine_sim[0, 1] 
            

        
print(compare_matrix)

          0         1         2         3         4         5         6   \
0   0.029359  0.005232  0.010091  0.017325  0.025636  0.014294  0.023087   
1   0.009582  0.011046  0.010416  0.015444  0.013668  0.015780  0.018655   
2   0.016815  0.014769  0.026587  0.010868  0.015350  0.024790  0.038619   
3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4   0.010567  0.001083  0.011695  0.010039  0.014469  0.021577  0.019111   
..       ...       ...       ...       ...       ...       ...       ...   
64  0.009778  0.005305  0.019099  0.028885  0.008401  0.030879  0.044503   
65  0.016279  0.008737  0.012869  0.015781  0.019460  0.030126  0.024146   
66  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
67  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
68  0.015708  0.012461  0.012017  0.019256  0.008788  0.023029  0.016292   

          7         8         9   ...        57        58        59        60  \
0   0.

In [48]:
compare_matrix[67] = compare_matrix.iloc[:, :67].sum(axis=1)

In [49]:
print(compare_matrix[67])

0     0.996059
1     0.997100
2     1.153962
3     0.000000
4     0.952554
        ...   
64    1.253020
65    1.050299
66    0.000000
67    0.000000
68    1.004078
Name: 67, Length: 69, dtype: float64


In [53]:
compare_matrix[67] = compare_matrix.iloc[:, :67].sum(axis=1)
print(compare_matrix[67])

sorted_compare_matrix = compare_matrix.sort_values(by=67, ascending=False) 
print(sorted_compare_matrix[67])

top_10_values = sorted_compare_matrix[67].head(10)

# Display the top 10 values
print("Top 10 values in column 67:")
print(top_10_values)

# Find the indices of the top 10 rows
top_10_indices = top_10_values.index

# Retrieve corresponding rows from moviesample
corresponding_movies = moviesample.iloc[top_10_indices]

# Print out the corresponding movie rows
print("\nCorresponding rows from moviesample:")
print(corresponding_movies)

64    1.253020
27    1.210767
20    1.197385
37    1.192447
46    1.191651
        ...   
29    0.000000
3     0.000000
66    0.000000
67    0.000000
15    0.000000
Name: 67, Length: 69, dtype: float64
Top 10 values in column 67:
64    1.253020
27    1.210767
20    1.197385
37    1.192447
46    1.191651
49    1.185327
13    1.180509
63    1.175110
9     1.171857
24    1.169978
Name: 67, dtype: float64

Corresponding rows from moviesample:
     movie_id                                       plot_summary  duration  \
64  tt0118749  Adult film director Jack Horner is always on t...  2h 35min   
27  tt0177789  The sci-fi television series "Galaxy Quest", w...  1h 42min   
20  tt0758758  Based on a true story. After graduating from E...  2h 28min   
37  tt2096673  Growing up can be a bumpy road, and it's no ex...  1h 35min   
46  tt1371111  Everything is connected: an 1849 diary of an o...  2h 52min   
49  tt0200550  Sexy, romantic comedy about a girl in her earl...  1h 40min   
13  tt01024

In [54]:
## By jaccard_distance

from nltk.metrics.distance import jaccard_distance

compare_matrix2 = pd.DataFrame(0, index=range(num_movies), columns=range(num_animes))

for i in range(num_movies):
    for j in range(num_animes):
        movietext = moviesample['plot_synopsis'][i]
        animetext = anime_fire1['synopsis'][j]
        
        if pd.isna(movietext) or pd.isna(animetext):
             compare_matrix2.iat[i, j] = 0
        else:
            j_dist = 1-jaccard_distance(set(movietext),set(animetext))

            # Store the similarity score in the compare_matrix
            compare_matrix2.iat[i, j] = cosine_sim[0, 1] 
            

        
print(compare_matrix2)

          0         1         2         3         4         5         6   \
0   0.014965  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965   
1   0.014965  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965   
2   0.014965  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965   
3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4   0.014965  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965   
..       ...       ...       ...       ...       ...       ...       ...   
64  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965   
65  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965   
66  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
67  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
68  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965  0.014965   

          7         8         9   ...        57        58        59        60  \
0   0.

In [56]:
compare_matrix2[67] = compare_matrix2.iloc[:, :67].sum(axis=1)
print(compare_matrix[67])

sorted_compare_matrix2 = compare_matrix2.sort_values(by=67, ascending=False) 
print(sorted_compare_matrix[67])

top_10_values = sorted_compare_matrix2[67].head(10)

print(top_10_values)


top_10_indices = top_10_values.index

corresponding_movies = moviesample.iloc[top_10_indices]

print("\nCorresponding rows from moviesample:")
print(corresponding_movies)

0     0.996059
1     0.997100
2     1.153962
3     0.000000
4     0.952554
        ...   
64    1.253020
65    1.050299
66    0.000000
67    0.000000
68    1.004078
Name: 67, Length: 69, dtype: float64
64    1.253020
27    1.210767
20    1.197385
37    1.192447
46    1.191651
        ...   
29    0.000000
3     0.000000
66    0.000000
67    0.000000
15    0.000000
Name: 67, Length: 69, dtype: float64
Top 10 values in column 67:
0     1.002668
30    1.002668
1     1.002668
35    1.002668
36    1.002668
37    1.002668
38    1.002668
39    1.002668
40    1.002668
44    1.002668
Name: 67, dtype: float64

Corresponding rows from moviesample:
     movie_id                                       plot_summary  duration  \
0   tt2294449  After making their way through high school (tw...  1h 52min   
30  tt0286716  Bruce Banner, a brilliant scientist with a clo...  2h 18min   
1   tt1951264  Twelve months after winning the 74th Hunger Ga...  2h 26min   
35  tt0209163  Many years ago, in Ancient E

In [63]:
import spacy

Dragon_Cry = fire_mc_synopsis['Fairy Tail Movie 2: Dragon Cry'][0]
Phoenix_Priestess = fire_mc_synopsis['Fairy Tail Movie 1: Houou no Miko'][0]

nlp = spacy.load("en_core_web_sm")

text1 = nlp(Dragon_Cry)
text2 = nlp(Phoenix_Priestess)

# Compare similarity
similarity = text1.similarity(text2)
print(f"Similarity: {similarity}")

Similarity: 0.9407522082328796


In [65]:
import spacy

compare_matrix3 = pd.DataFrame(0, index=range(num_movies), columns=range(num_animes))
nlp = spacy.load("en_core_web_sm")

for i in range(num_movies):
    print("Movie num " + str(i))
    for j in range(num_animes):
        movietext = moviesample['plot_synopsis'][i]
        animetext = anime_fire1['synopsis'][j]
        
        if pd.isna(movietext) or pd.isna(animetext):
             compare_matrix3.iat[i, j] = 0
        else:
            movienlp = nlp(movietext)
            animenlp = nlp(animetext)
            spacy_similarity = movienlp.similarity(animenlp)
            compare_matrix3.iat[i, j] = spacy_similarity
            

        
print(compare_matrix3)

Movie num 0
Movie num 1
Movie num 2
Movie num 3
Movie num 4
Movie num 5
Movie num 6
Movie num 7
Movie num 8
Movie num 9
Movie num 10
Movie num 11
Movie num 12
Movie num 13
Movie num 14
Movie num 15
Movie num 16
Movie num 17
Movie num 18
Movie num 19
Movie num 20
Movie num 21
Movie num 22
Movie num 23
Movie num 24
Movie num 25
Movie num 26
Movie num 27
Movie num 28
Movie num 29
Movie num 30
Movie num 31
Movie num 32
Movie num 33
Movie num 34
Movie num 35
Movie num 36
Movie num 37
Movie num 38
Movie num 39
Movie num 40
Movie num 41
Movie num 42
Movie num 43
Movie num 44
Movie num 45
Movie num 46
Movie num 47
Movie num 48
Movie num 49
Movie num 50
Movie num 51
Movie num 52
Movie num 53
Movie num 54
Movie num 55
Movie num 56
Movie num 57
Movie num 58
Movie num 59
Movie num 60
Movie num 61
Movie num 62
Movie num 63
Movie num 64
Movie num 65
Movie num 66
Movie num 67
Movie num 68
          0         1         2         3         4         5         6   \
0   0.589116  0.556957  0.521218  0.6

In [66]:
compare_matrix3[67] = compare_matrix3.iloc[:, :67].sum(axis=1)
print(compare_matrix3[67])

sorted_compare_matrix3 = compare_matrix3.sort_values(by=67, ascending=False) 
print(sorted_compare_matrix3[67])

top_10_values = sorted_compare_matrix3[67].head(10)

# Display the top 10 values
print("Top 10 values in column 67:")
print(top_10_values)

# Find the indices of the top 10 rows
top_10_indices = top_10_values.index

# Retrieve corresponding rows from moviesample
corresponding_movies = moviesample.iloc[top_10_indices]

# Print out the corresponding movie rows
print("\nCorresponding rows from moviesample:")
print(corresponding_movies)

0     37.207724
1     39.411485
2     34.403477
3      0.000000
4     38.544541
        ...    
64    41.245789
65    37.433338
66     0.000000
67     0.000000
68    40.959453
Name: 67, Length: 69, dtype: float64
51    46.684867
62    45.131380
64    41.245789
54    41.006820
68    40.959453
        ...    
42     0.000000
43     0.000000
45     0.000000
12     0.000000
58     0.000000
Name: 67, Length: 69, dtype: float64
Top 10 values in column 67:
51    46.684867
62    45.131380
64    41.245789
54    41.006820
68    40.959453
44    40.621142
13    40.555778
48    40.487422
11    40.305045
57    40.263599
Name: 67, dtype: float64

Corresponding rows from moviesample:
     movie_id                                       plot_summary  duration  \
51  tt0388980  Near the turn of the twentieth century, young ...        2h   
62  tt0450314  Frank Castle, the ex-military man whose family...  1h 43min   
64  tt0118749  Adult film director Jack Horner is always on t...  2h 35min   
54  tt03647